In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging
from collections import defaultdict

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getSuperCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [9]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
#             if v - map1[k] != 0:
            map3[k] = v - map1[k]
    return map3

In [10]:
def diffAndFilter2(map1, map2):
    map3 = dict()
    for key, val in map2.items():
        diff = dict()
        for k, v in map2[key].items():
#             if v - map1[key][k] != 0:
            diff[k] = v - map1[key][k]
        map3[key] = diff
    return map3

In [11]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [12]:
def R5_test(single):
    logger.info(f"Test-R5: Reward given out to block signers sums up to the total block reward")
    logger.info(f"check sum over earned-reward diff of all keys per shard = 28")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
                    
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    time.sleep(3)
    logger.info(f"current block {block}, will begin collecting infos...")

    earn_rewards_prev = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[shard][bls_key] = earn_reward
                
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    time.sleep(3)
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[shard][bls_key] = earn_reward
    earn_reward_diff = diffAndFilter2(earn_rewards_prev, earn_rewards_curr) 
    for shard, diff in earn_reward_diff.items():
        reward = sum(diff.values())
        if format(reward, '.20e') != format(block_reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"shard: {shard}, block: {block}, sum of earned reward: {reward:.20e}, block reward: {block_reward:.20e}\n")
            flag = False
    
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [13]:
R5_test(True)

INFO:economic-test:Test-R5: Reward given out to block signers sums up to the total block reward
INFO:economic-test:check sum over earned-reward diff of all keys per shard = 28
INFO:economic-test:current and last block numbers: 2663, 2699


current and last block numbers 2663 2699


INFO:economic-test:current block 2664, will begin collecting infos...
INFO:economic-test:new block 2665 reached, will begin testing...
INFO:economic-test:Test-R5: Succeed



(True, None)